<a href="https://colab.research.google.com/github/CT-Cultures/Content/blob/master/RegOverview_Issue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount drive and install requirements

In [ ]:
# Mount Drive
import os
import sys
import gc

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install requirements in this cell,
# then restart runtime after initial installation

# fetch Content from github and install requirements
path_Content = "/content/drive/MyDrive/Github/Content" ###
if not os.path.exists(path_Content):
  !git clone https://github.com/CT-Cultures/Content.git {path_Content}
os.chdir(path_Content)
!pip install -r requirements.txt

path_Article = "/content/drive/Mydrive/Github/Article" ###
if not os.path.exists(path_Article):
  !git clone https://github.com/CT-Cultures/Article.git {path_Article}

In [ ]:
# Check Environment
import pandas as pd
import transformers
import nltk
print('pandas version: {}, (>= 1.3.2)'.format(pd.__version__)) # pd has to >= 1.3.2, restart runtime
print('transformers version: {}'.format(transformers.__version__))
#print('nltk version: {}, (>=3.3)'.format(nltk.__version__))

!which python
!python --version
!nvidia-smi

In [ ]:
%%capture
!pip install -r sources/ChinaFilm/requirements.txt
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

!pip install transformers

# Import libraries and set path

In [ ]:
#Load Libraries Global
import os
import datetime as dt
import re
import pandas as pd
import numpy as np
import torch
from bs4 import BeautifulSoup

from selenium import webdriver

import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.font_manager as fm

path_fonts = '/content/drive/MyDrive/Github/Article/fonts'
fontprop = fm.FontProperties(fname=path_fonts, size= 15)

font_dirs = [path_fonts, ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)
font_list = fm.createFontList(font_files)
for font in font_files:
  fm.fontManager.addfont(font)

plt.rcParams['figure.figsize'] = [15, 9]
mp.rcParams['font.family'] = ['Microsoft YaHei']

%matplotlib inline
print(mp.get_cachedir())

In [ ]:
# Set Path
path_wd = path_Article = '/content/drive/MyDrive/Github/Article'
path_Article_records = '/content/drive/MyDrive/Github/Article/records'

path_font = '/content/drive/MyDrive/Github/Article/fonts/STHUPO.TTF'
path_img = '/content/drive/MyDrive/Github/Article/img'

path_Content = '/content/drive/MyDrive/Github/Content'
path_NRTA = path_Content + '/tools/sources/NRTA'

os.chdir(path_wd)

In [ ]:
# Load Local Lib# Set Path

# import utils for making plots and wordclouds
os.chdir(path_wd)
from generate import utils

# import tools for prediction
os.chdir(path_Content)
import predict

# intantiate tv_reg
os.chdir(path_NRTA)
from Record_Registration import Registration # assume relative path at path_wd
tv_reg = Registration()

## 1.3 Load Latest Registration Publication

In [ ]:
fp = path_NRTA + '/records/contents_of_registrations.json'
df = pd.read_json(fp)

In [ ]:
curr_issue_dt = df['公示年月'].iloc[0]
issue_name = str(curr_issue_dt)
issue_name

In [ ]:
# clean df, run mulitple times as necessary
df['许可证号'] = df['许可证号'].apply(lambda x:
                    x.lstrip('\n+').lstrip('\t+').lstrip('\w+').rstrip('\t+').rstrip('\n+'))
df['备注'] = df['备注'].apply(lambda x:
                    x.lstrip('\n+').lstrip('\t+').lstrip('\w+').rstrip('\t+').rstrip('\n+'))

In [ ]:
df_curr = df[df['公示年月'] == curr_issue_dt].copy()
print(df_curr.columns)

#[2] Load Model for Predictons


## [2.1] Extract top keywords

In [ ]:
df_curr['kw'] = df_curr['内容提要'].apply(textrank, topK=10)

In [ ]:
df_curr['kw'].to_numpy()

## [2.2] Load Title Prediction Model

In [ ]:
%%capture
from transformers import BertTokenizer, BartForConditionalGeneration

# assign device
if torch.cuda.device_count() > 0:
  device = 'cuda:' + str(torch.cuda.current_device())
else:
  device = 'cpu'

# Instantiate tokenizer and model
checkpoint = "/content/drive/MyDrive/Github/Content/tools/models/PredTitle-10000"

tokenizer = BertTokenizer.from_pretrained(checkpoint)
model = BartForConditionalGeneration.from_pretrained(checkpoint)
model.to(device)
model.eval()

In [ ]:
batch_size = 8
i = 0
ls = df_curr['内容提要'].tolist()
L = df_curr.shape[0]
test_predictions = []

while i < L:
  inputs = tokenizer(ls[i:i+batch_size],
                           padding=True,
                           max_length=512, 
                           truncation=True, 
                           return_tensors='pt')
  inputs.to(device)
  summary_ids = model.generate(input_ids=inputs['input_ids'],
                             num_beams=4,
                             min_length=0,
                             max_length=32
                             )
  
  ret = [tokenizer.decode(g, 
                         skip_specical_tokens=True, 
                         clean_up_tokenization_spaces=True) for g in summary_ids]
  test_predictions.extend(ret)
  i += batch_size

In [ ]:
df_curr['预测剧名'] = test_predictions

def remove_specials(x):
  x = re.sub(' ', '', x)
  x = re.sub('\[CLS\]', '', x)
  x = re.sub('\[PAD\]', '', x)
  x = re.sub('\[SEP\]', '', x)
  x = re.sub('\[UNK\]', '', x)
  return x

df_curr['预测剧名'] = df_curr['预测剧名'].apply(remove_specials)

In [ ]:
df_curr['预测剧名'].to_numpy()

## [2.3] 识别主要角色

In [ ]:
df_curr['time'] = df_curr['题材'].apply(lambda x: x[0:2])
df_curr['genre'] = df_curr['题材'].apply(lambda x: x[2:])

df_curr['制作周期_月'] = df_curr['制作周期'].apply(lambda x: int(x.rstrip('个月')))

def months_to_principal_photography(x: pd.Series):
  year = int(x.split('.')[0])
  month = int(x.split('.')[1])
  now = datetime.datetime.now()
  p_date = datetime.date(year, month, 15)
  months_from_now = (p_date - now.date()).days // 30
  return months_from_now

df_curr['距离开机'] = df_curr['拍摄日期'].apply(months_to_principal_photography)
#df_curr['预估拍摄速度'] = df_curr['制作周期_月'] / df_curr['集数']*30

In [ ]:
import jieba
import jieba.posseg as pseg
jieba.enable_paddle() #启动paddle模式。 0.40版之后开始支持，早期版本不支持

In [ ]:
# 识别主要角色
def find_PER(x: pd.Series):
  words = pseg.cut(x, use_paddle=True)
  ls_PER = []
  for word, flag in words:
    if flag == 'PER':
      word = word.strip('[\s,，。、“”？；]')
      ls_PER.append(word)
  ls_PER = list(set(ls_PER))

  ls_PER1 = ls_PER.copy()
  ls_PER = []
  while ls_PER1:
    PER =  ls_PER1.pop(0)
    ls_tmp = re.split('[\s,，。、“”？；]', PER)
    ls_PER.extend(ls_tmp)

  ls_PER = list(set(ls_PER)) 
  ls_PER = [PER for PER in ls_PER if len(PER) > 1]
  ls_PER_sorted = sorted(ls_PER, reverse=False, key=lambda x: len(x))

  ans = []
  while ls_PER_sorted:
    PER = ls_PER_sorted.pop(0)
    i = 0
    while i < len(ls_PER_sorted):
      if re.search(PER, ls_PER_sorted[i]):
        ls_PER_sorted.pop(i)
        ans.append(PER)
      i += 1
    if PER not in ans:
      ans.append(PER)

  return ans

df_curr['主要角色'] = df_curr['内容提要'].apply(find_PER)

In [ ]:
df_curr['主要角色'].to_numpy()

## 调整类型

In [ ]:
df_curr['类型'] = df_curr['genre'].copy()

In [ ]:
df_curr['genre'].to_numpy()

In [ ]:
genre_v2 = ['喜剧', '爱情', '动作', '犯罪', '科幻', '奇幻', '冒险', '灾难', '恐怖',
 '惊悚', '剧情', '战争', '歌舞', '悬疑', '动画', '同性']

genre_v2 +=  ['商战', '青少', '家庭', '宅斗', '战争', '体育', '谍战', '涉案', '军旅', '职业', '传奇', '宫斗']
genre_v2 +=  ['职业', '商战', '体育', '涉案']
genre_v2 +=  ['科幻', '奇幻', '冒险', '神话', '仙侠']
genre_v2 +=  ['历史', '宫廷', '宫斗', '宅斗', '传奇']
genre_v2 +=  ['革命', '战争', '军旅']
genre_v2 +=  ['都市', '农村']
genre_v2 +=  ['爱情', '喜剧', '惊悚', '恐怖', '歌舞']
genre_v2 +=  ['灾难']
genre_v2 +=  ['青少']
genre_v2 +=  ['其它']

genre_v2 = list(set(genre_v2))

In [ ]:
df_curr['类型'] = ['涉案', '爱情', '家庭', '爱情', '革命', '爱情', '革命', '职业', '涉案', '奋斗', '家庭',
       '奋斗', '农村', '农村', '传奇', '革命', '奋斗', '家庭', '奇幻', '剧情', '家庭', '奋斗',
       '家庭', '涉案', '奋斗', '剧情', '传奇', '商战', '奋斗', '家庭', '农村', '商战', '传奇',
       '商战', '农村', '职业', '农村', '传记', '革命']

In [ ]:
HTML(df_curr[['剧名', 'genre', '类型', '内容提要']].to_html())

# [2] Generating Article Contents

##[2.1] 2021年X月电视剧备案划重点

In [ ]:
# Title
T1 = '{}电视剧备案划重点'.format(df_curr['公示年月'].iloc[0])

print(T1)

In [ ]:
dfview = df_curr[[
                   '剧名', '集数', '距离开机', '类型', 
                   'time', '主要角色', '报备机构', '内容提要',
                   '预测剧名', 'kw']].sort_values('距离开机')
dfview.info()

## 按类型划分

In [ ]:
df_by_genre_episode = \
  dfview.groupby('类型')['集数'].sum().rename('nepisodes'
  ).reset_index().sort_values('nepisodes', ascending=False)

plt.clf()
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['axes.facecolor'] = 'white'
ax = df_by_genre_episode.plot(
    kind = 'bar',
    grid = True,
    fontsize = 22,
    rot = 0,
    color = ['violet'],
)
ax.set_title("按类型划分",fontsize= 24, pad=20)
ax.spines['top'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')
ax.spines['right'].set_color('black')
ax.grid(color='gray', linestyle='-', linewidth=0.5)
ax.set_xlabel('类型',fontsize= 18)
ax.set_xticklabels(df_by_genre_episode['类型'], fontsize=12)
ax.set_ylabel("集数",fontsize= 18)
ax.legend(fontsize=22)

fp_plot_time = path_img + '/df_TVReg_plot_genre_by_nepisodes_{}.png'.format(issue_name)
plt.savefig(fp_plot_time, bbox_inches='tight')

plt.show()

In [ ]:
# WIP
df_by_genre= dfview.groupby('类型')['类型'].count(
    ).rename('数量').reset_index().sort_values('数量', ascending=False)

plt.clf()
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['axes.facecolor'] = 'white'
ax = df_by_genre.plot(
    kind = 'bar',
    grid = True,
    fontsize = 22,
    rot = 0,
    color = ['violet'],
)
ax.set_title("按类型划分",fontsize= 24, pad=20)
ax.spines['top'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')
ax.spines['right'].set_color('black')
ax.grid(color='gray', linestyle='-', linewidth=0.5)
ax.set_xlabel('类型',fontsize= 18)
ax.set_xticklabels(df_by_genre['类型'], fontsize=12)
ax.set_ylabel("数量",fontsize= 18)
ax.legend(fontsize=22)

fp_plot_time = path_img + '/df_TVReg_plot_genre_by_n_{}.png'.format(issue_name)
plt.savefig(fp_plot_time, bbox_inches='tight')

plt.show()

##[2.2] 	本期通过备案的电视剧共计39部,估计已开机的1部,一个月内将开机的20部,距离开机一个月以上的18部。其中，20集(含）以下的电视剧又4部,20到40集的有35部,超过40集的有0部。


In [ ]:
dfview.columns
dfview['集数'] = dfview['集数'].astype('int')

In [ ]:
# Write Content
T2 = '\n'
T2 += '本期通过备案的电视剧共计{}部，'.format(dfview.shape[0])
T2 += '估计已开机的{}部，'.format(dfview[dfview['距离开机'] < 0].shape[0])
T2 += '一个月内将开机的{}部，'.format(
    dfview[(dfview['距离开机'] >= 0) & (dfview['距离开机'] < 2)].shape[0])

T2 += '距离开机一个月以上的{}部。'.format(
    dfview[(dfview['距离开机'] > 2)].shape[0])

T2 += '其中，20集(含）以下的电视剧有{}部，'.format(dfview[dfview['集数'] <= 20].shape[0])
T2 += '20到40集的有{}部，'.format(
    dfview[(dfview['集数'] > 20) & (dfview['集数'] <= 40)].shape[0])
T2 += '超过40集的有{}部。'.format(dfview[dfview['集数'] > 40].shape[0])

print(T2)

##[2.3] 	本批次中，大数据分析识别出XX部上市影视公司关联项目，占比XX%。慷田AI聚焦关注的有

In [ ]:
HTML(df_curr[['剧名', '题材', '报备机构', '内容提要']].to_html())

In [ ]:
df_curr['报备机构'].unique()

In [ ]:
# 本批次中与上市影视公司关联的项目有
info_public_film_co = pd.read_csv('/content/drive/MyDrive/Github/Article/reference/info_public_film_co.csv', index_col=0, encoding='utf-8-sig')
ls = info_public_film_co['公司简称'].apply(eval).sum()
pat_public = '|'.join(ls)
pat_public += '|阿里|腾讯|爱奇艺|英皇|寰亚|银都|美亚|大盛|儒意|灿星|横店|华策|电视剧制作中心'
pat_public += '|得闲|芒果|新丽|欢乐|尚世|华策|稻草熊|东阳欢娱|耀客|湖南快乐阳光|山东影视制作|当代时光'
pat_public += '|唐德|欢瑞|优酷|嘉行|东阳欢愉|稻草熊|天马映像'
df_focus = dfview.loc[dfview['报备机构'].str.contains(pat_public), :]
df_focus[['剧名','报备机构', '集数','内容提要', '类型']]

In [ ]:
ids = [5,20,32,18]
df_focus_narrowed = df_focus.loc[ids]

In [ ]:
df_focus_narrowed['单位简称'] = df_focus_narrowed['报备机构'].str.extract('('+ pat_public + ")")

In [ ]:
df_focus_narrowed.info()

In [ ]:
########
T3 = '\n'
T3 += '本批次中，ContentAI识别出{}部上市影视公司及国资参投影视公司关联项目，'.format(df_focus.shape[0])
T3 += '占比{}%。'.format(round((df_focus.shape[0]/df_curr.shape[0]*100),2))
T3 += '结合题材与出品方实力，ContentAI聚焦关注的有'

for i, row in df_focus_narrowed.iterrows():
  T3 += '{}的'.format(row['单位简称'])
  if i == df_focus_narrowed.index[-2]:
    T3 += '《{}》和'.format(row['剧名'])
  else:
    T3 += '《{}》、'.format(row['剧名'])
T3 = T3.rstrip('、') + '，'

T3 += '题材类型包括了{}。\n'.format(
    '、'.join(df_focus_narrowed['类型'].unique())
)
print(T3)

In [ ]:
df_focus_narrowed.index[-2]

## [2.4] 生成词云图

In [ ]:
df_focus_narrowed

In [ ]:
os.chdir(path_wd)
#%load_ext autoreload
%reload_ext autoreload
from generate import utils

from IPython.display import Image as Img
from PIL import Image as pil
path_img = '/content/drive/MyDrive/Github/Article/img'

In [ ]:
from IPython.display import Image as Img
from PIL import Image as pil

path_posters = '/content/drive/MyDrive/Github/Article/img/posters'
path_icon = '/content/drive/MyDrive/Github/Article/img/genre_icon'

#df_label2image.to_json(path_records + '/df_label2image.json')
df_label2image = pd.read_json(path_wd + '/records/df_label2img.json')

In [ ]:
df_label2image.loc[27,:] = ['商战', 'swan-46510_1280.png']
df_label2image.loc[28,:] = ['战争','explosion.png']
df_label2image.loc[29,:] = ['体育', 'goalkeeper-294327_1280.png']
df_label2image.loc[30,:] = ['谍战', 'butterfly-47967_1280.png']
df_label2image.loc[31,:] = ['奇幻', 'hibiscus-304330_1280.png']

In [ ]:
df_label2image

In [ ]:
df_focus_narrowed['src_img'] = None
df_focus_narrowed['tgt_img'] = None

In [ ]:
for i in df_focus_narrowed.index:
  ls = [df_focus_narrowed.loc[i, '剧名']] * 10
  ls += [df_focus_narrowed.loc[i, '预测剧名']] *6
  ls += df_focus_narrowed.loc[i, '主要角色']*3
  ls += [df_focus_narrowed.loc[i, '类型']] *3
  ls += [df_focus_narrowed.loc[i, 'time']]*3
  ls += df_focus_narrowed.loc[i, 'kw']
  img_fn = df_focus_narrowed.loc[i, 'src_img']
  if not img_fn:
    img_fn = df_label2image.loc[
        df_label2image['label'] == df_focus_narrowed.loc[i, '类型'],
        'fn'
    ].iloc[0]
  #print(txt)
  fp_img = path_img + '/genre_icon/{}'.format(img_fn)
  fp_mask = path_img + '/genre_icon/{}'.format('mask_' + img_fn)
  #fp_img = path_img + '/genre_icon/{}'.format('psychedelic-1084082_960_720.jpg')
  fp_generated_img = utils.generate_word_image(ls, 
                                               fp_img, 
                                               fp_mask,
                                               fp_prefix='NRTA_TVReg_Overview',
                                               fp_suffix=issue_name,
                                               img_width=400,
                                               )
  #display(Img(fp_generated_img, width=400))
  df_focus_narrowed.loc[i, 'tgt_img'] = fp_generated_img

##[2.5] 生成摘要

In [ ]:
S0 = issue_name
S0 += '电视剧备案慷田AI聚焦关注的有'

for i, row in df_focus_narrowed.iterrows():
  S0 += '{}的'.format(row['单位简称'])
  if i == df_focus_narrowed.index[-2]:
    S0 += '《{}》和'.format(row['剧名'])
  else:
    S0 += '《{}》、'.format(row['剧名'])
S0 = S0.rstrip('、') + '，'

S0 += '题材类型包括了{}。\n'.format(
    '、'.join(df_focus_narrowed['类型'].unique())
)
print(S0)

##[2.6] 指向国家广电局官网


In [ ]:
#####
R1 = '\n\n'
R1 += 'ContentAI结合自主调研及多方大数据比对，通过分析、建模，提炼关键信息。'
R1 += '电视剧备案公示信息来自国家广播电视总局 National Radio and Televison Administration, 官方网址 '
R1 += ' http://www.nrta.gov.cn/ 。'

print(R1)

In [ ]:
R2 = '点击左下角阅读原文查看本期ContentAI电视剧信息详表。'
R3 = '点击左下角阅读原文查看本期ContentAI电视剧概览分析。'

In [ ]:
issue_name

## [2.7] Save df of this issue pickle

In [ ]:
df_curr.to_pickle(path_records + '/df_tvreg_{}.pkl'.format(issue_name))
df_focus_narrowed.to_pickle(path_records + '/df_tvreg_focus_{}.pkl'.format(issue_name))

In [ ]:
df_curr = pd.read_pickle(path_records + '/df_tvreg_{}.pkl'.format(issue_name))
df_focus_narrowed = pd.read_pickle(path_records + '/df_tvreg_focus_{}.pkl'.format(issue_name))

#[3] Output Word Document

## 3.1 Install and Load Libraries

In [ ]:
%%capture
!pip install python-docx
!pip install lxml
from docx import Document
from docx.shared import Inches
#from docx.text.parargaph import Paragraph

In [ ]:
df_focus_narrowed[['报备机构']]

## 3.2 Output Docx


In [ ]:
from docx import Document
from docx.shared import Inches
from docx.oxml.ns import qn

### Write Overview docx

In [ ]:
path_doc = '/content/drive/MyDrive/Github/Article/docx'
path_img = '/content/drive/MyDrive/Github/Article/img'

doc = Document()

# Set document Font 
doc.styles['Normal'].font.name = '微软雅黑'
r = doc.styles['Normal']._element
r.rPr.rFonts.set(qn('w:eastAsia'), '微软雅黑')

# Set Document Title
doc.core_properties.title = 'China TV Registration Overview ' + issue_name

doc.add_heading(T1, 0)

p = doc.add_paragraph(S0, style='Intense Quote')

doc.add_paragraph(T2)

doc.add_picture(fp_plot_time, width=Inches(6))
doc.add_paragraph(T3)

for i, row in df_focus_narrowed.iterrows():
  if  row['距离开机'] <= 0: pstatus = '估摸着已开机'
  elif  0 < row['距离开机'] <= 1: pstatus = '预计一个月内开机'
  else: pstatus = '预计距离开机1个月以上' 
  txt = '\n《{}》，{}集，{}。'.format(row['剧名'], row['集数'], pstatus)
  txt += '\n报备机构：{}'.format(row['报备机构'])
  txt += '\n主要角色：{}'.format('、'.join(row['主要角色']))
  doc.add_paragraph(txt)
  doc.add_picture(row['tgt_img'], width =Inches(4))
  doc.add_paragraph(row['内容提要'])

doc.add_paragraph(R1)
doc.add_paragraph(R2)

fp_doc = path_doc + '/TVregHighlight_' + issue_name + '.docx'
doc.save(fp_doc)


In [ ]:
# Download Document
from google.colab import files
files.download(fp_doc)

### write Table docx

In [ ]:
dfview.columns

In [ ]:
doc = Document()

# Set document Font 
doc.styles['Normal'].font.name = '微软雅黑'
r = doc.styles['Normal']._element
r.rPr.rFonts.set(qn('w:eastAsia'), '微软雅黑')

# Set Document Title
doc.core_properties.title = 'China TV Registration Table ' + issue_name

doc.add_heading('详表：{}'.format(T1), 0)

p = doc.add_paragraph(S0, style='Intense Quote')

table = doc.add_table(rows=1, cols=1, style='Light List Accent 3')
i = 1
for _, row in dfview.iterrows():
  row_cells = table.add_row().cells
  row_cells[0].text = '[{}] '.format(i) + \
                      '《{}》，'.format(row['剧名']) + \
                      '{}集'.format(row['集数'])

  #if row['匹配片名'] != '':
  #  row_cells[0].text += ' (原备案名《{}》)'.format(row['匹配片名'])

  row_cells = table.add_row().cells
  row_cells[0].text = row['time'] + row['类型']
  
  if  row['距离开机'] <= 0: pstatus = '估摸着已开机'
  elif  0 < row['距离开机'] <= 1: pstatus = '预计一个月内开机'
  else: pstatus = '预计{}个月后开机'.format(row['距离开机'])
  row_cells = table.add_row().cells
  row_cells[0].text = pstatus
  #if row['是否修改'] == '是':
  #  row_cells[0].text += ', 修改后通过备案'

  row_cells = table.add_row().cells
  row_cells[0].text = '报备机构: {}'.format(row['报备机构'])

  row_cells = table.add_row().cells
  row_cells[0].text = '主要角色: {}'.format('、'.join(row['主要角色']))

  row_cells = table.add_row().cells
  row_cells[0].text = row['内容提要']

  row_cells = table.add_row().cells
  row_cells[0].text = ''
  i+=1

doc.add_paragraph(R1)
doc.add_paragraph(R3)

fp_doc = path_doc + '/TVregTable_' + issue_name + '.docx'
doc.save(fp_doc)


In [ ]:
# Download Document
from google.colab import files
files.download(fp_doc)